### YOLO9000: Better, Faster, Stronger

### Darknet-19每个卷积层后面同样使用了batch norm层以加快收敛速度，降低模型过拟合

<IMG SRC="0825/3.JPG">

<img src="0825/4.jpg">

<img src="0825/5.jpg">

### YOLOv2增加了在ImageNet数据集上使用448*448,来finetune分类网络这一中间过程（10 epochs），这可以使得模型在检测数据集上finetune之前已经适用高分辨率输入

### YOLOv2借鉴了Faster R-CNN中RPN网络的先验框（anchor boxes，prior boxes，SSD也采用了先验框）策略。RPN对CNN特征提取器得到的特征图（feature map）进行卷积来预测每个位置的边界框以及置信度（是否含有物体），并且各个位置设置不同尺度和比例的先验框，所以RPN预测的是边界框相对于先验框的offsets值（其实是transform值，详细见Faster R_CNN论文），采用先验框使得模型更容易学习

### YOLOv2移除了YOLOv1中的全连接层而采用了卷积和anchor boxes来预测边界框。为了使检测所用的特征图分辨率更高，移除其中的一个pool层。在检测模型中，YOLOv2不是采418 * 418图片作为输入，而是采用416 * 416大小。因为YOLOv2模型下采样的总步长为32,对于416 * 416大小的图片，最终得到的特征图大小为13 * 13，维度是奇数，这样特征图恰好只有一个中心位置。对于一些大物体，它们中心点往往落入图片中心位置，此时使用特征图的一个中心点去预测这些物体的边界框相对容易些。所以在YOLOv2设计中要保证最终的特征图有奇数个位置。

### YOLOv2使用了anchor boxes之后，每个位置的各个anchor box都单独预测一套分类概率值，这和SSD比较类似（但SSD没有预测置信度，而是把background作为一个类别来处理）

### YOLOv2采用k-means聚类方法对训练集中的边界框做了聚类分析。因为设置先验框的主要目的是为了使得预测框与ground truth的IOU更好，所以聚类分析时选用box与聚类中心box之间的IOU值作为距离指标

### 最终选取5个聚类中心作为先验框

### 预测边界框中心点相对于对应cell左上角位置的相对偏移值，为了将边界框中心点约束在当前cell中，使用sigmoid函数处理偏移值，这样预测的偏移值在(0,1)范围内（每个cell的尺度看做1）

### 可以将边界框相对于整张图片的位置和大小计算出来（4个值均在0和1之间）,再将上面的4个值分别乘以图片的宽度和长度（像素点值）就可以得到边界框的最终位置和大小了

## YOLOv2的一大创新是采用Multi-Scale Training策略

### 尽管YOLOv2和YOLOv1计算loss处理上有不同，但都是采用均方差来计算loss